In [ ]:
# import modules and define functions
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt, ticker, cm
import matplotlib as mpl
import os

In [ ]:
outfs_dimer = [i for i in os.listdir(os.getcwd()) if i.endswith('.out') and 'H2' in i]
outfs_dimer += os.path.join('outputs', 'H2_0_1_FCI_def2-qzvpp.sp.out')
outfs_mononer = [i for i in os.listdir(os.getcwd()) if i.endswith('.out') and 'H_' in i]
data_dimer_dict = {}
data_monomer_dict = {}
for outf in outfs_dimer:
    mult = outf.split('_')[2]
    if 'B3LYP' in outf:
        sd = outf.split('_')[3][0] + 'KS'
    else:
        sd = outf.split('_')[3]
    sd = "%s%s" %(mult, sd)
    lines = open(outf, 'r').read().splitlines()
    s2s = []
    for i, line in enumerate(lines):
        if '%paras' in line:
            nstep = 20
        if 'UKS' in sd:
            if 'Expectation value of <S**2>' in line:
                s2 = float(list(filter(None,line.split(' ')))[-1])
                s2s.append(s2)
        if "The Calculated Surface using the 'Actual Energy'" in line:
            data = lines[i+1:i+1+nstep]
            data = [list(filter(None, i.split(' '))) for i in data]
            keys, es = np.array(data, dtype=float).T
            break
    if 'UKS' in sd:
        data_dimer_dict[sd] = np.array([es,s2s[-nstep:]])
    else:
        data_dimer_dict[sd] = np.array(es)

for outf in outfs_mononer:
    lines = open(outf, 'r').read().splitlines()
    for i, line in enumerate(lines):
        if 'FINAL SINGLE' in line:
            e_monomer = float(list(filter(None, line.split(' ')))[4])
            break

data_dict = {}
for key in data_dimer_dict.keys():
    if 'UKS' in key:
        data_dict[key] = data_dimer_dict[key][0] - np.array(e_monomer) * 2
    else:
        data_dict[key] = data_dimer_dict[key] - np.array(e_monomer) * 2
data_dict['fsc'] = data_dimer_dict['1UKS'][1]/(data_dimer_dict['3UKS'][1] - data_dimer_dict['1UKS'][1])
data_dict['1UKS_pure'] = data_dimer_dict['1UKS'][0] + data_dict['fsc'] * (data_dimer_dict['1UKS'][0] - data_dimer_dict['3UKS'][0]) - np.array(e_monomer) * 2
    

In [ ]:
df = pd.DataFrame(data_dict, dtype=float)
df.index = keys

In [ ]:
# plot the change of energy 
xlabel = 'H-H distance / A'
ylabel = 'Dissociation energy / Eh'
ylabel2 = 'Fraction of spin contamination'
title = 'H2 stretching'
mpl.rcParams['axes.linewidth'] = 3
mpl.rcParams['xtick.major.size'] = 10
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16
mpl.rcParams['xtick.major.width'] = 3
mpl.rcParams['ytick.major.size'] = 10
mpl.rcParams['ytick.major.width'] = 3
mpl.rcParams['xtick.minor.size'] = 5
mpl.rcParams['xtick.minor.width'] = 3
mpl.rcParams['ytick.minor.size'] = 5
mpl.rcParams['ytick.minor.width'] = 3
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'

# plotting
fig, ax = plt.subplots(figsize=(7,6))
ax2 = ax.twinx()
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(0.5))
ax.yaxis.set_major_locator(ticker.MultipleLocator(0.1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.05))
ax.set_ylim(-0.2, 0.1)
ax.set_xlim(0,5)
ax.set_xlabel(xlabel, fontsize=16)
ax.set_ylabel(ylabel, fontsize=16)
ax.set_title(title, fontsize=22)
ax2.yaxis.set_major_locator(ticker.MultipleLocator(0.25))
ax2.yaxis.set_minor_locator(ticker.MultipleLocator(0.125))
ax2.set_ylim(-0.1, 1.1)
ax2.set_xlabel(xlabel, fontsize=16)
ax2.set_ylabel(ylabel2, fontsize=16)
for key in df.columns: 
    if 'fsc' not in key:
        ax.plot(df.index, df[key], label=key, linestyle='dashed')
    # else:
    #     ax2.plot(df.index, df[key], label=key, linestyle='dashed')
ax.legend(loc='center right')
plt.savefig('H2_stretching_electronics_states.png', dpi=400)
# ax2.legend(loc='lower right')